In [2]:
import json

with open("MultiHopRAG.json", "r") as f:
    content = json.load(f)

In [5]:
import pandas as pd

df = pd.read_csv("multihoprag_corpus_summary.csv")
docs = list(df['content'])

In [21]:
start = 0
end = 50

content = content[start:end]

queries = [data['query'] for data in content]
len(queries)

50

In [22]:
answers = [data['answer'] for data in content]
len(answers)

50

In [23]:
gold_docs = []

for data in content:
    docs = []
    for evidence in data['evidence_list']:
        match = df.loc[df['title'] == evidence['title'], 'content']
        if not match.empty:
            docs.append(match.iloc[0])
        else:
            docs.append(None)  # or skip / handle as needed
    gold_docs.append(docs)

len(gold_docs)

50

In [15]:
gold_docs[2]

['Title:OpenAI\'s ex-chairman accuses board of going rogue in firing Altman: \'Sam and I are shocked and saddened by what the board did\'\nPassage:OpenAI fired CEO Sam Altman for being "not consistently candid" with the board, leading chairman Greg Brockman to quit. Brockman stated Altman was informed of his firing by Ilya Sutskever after being asked to join a video meeting with the other board members. Brockman added, "Sam and I are shocked and saddened by what the board did today," . OpenAI declined to elaborate on the reasons for Altman\'s dismissal. Altman expressed his love for his time at OpenAI, while Microsoft affirmed its commitment to its partnership with the company. The board consists of OpenAI’s chief scientist, Ilya Sutskever, and three non-employees: Quora CEO Adam D’Angelo, tech entrepreneur Tasha McCauley, and Helen Toner of the Georgetown Center for Security and Emerging Technology.',
 "Title:WTF is going on at OpenAI? We have theories\nPassage:Sam Altman was ousted a